# Challenge: QUBO


[<img src="https://qbraid-static.s3.amazonaws.com/logos/Launch_on_qBraid_white.png" width="150">](https://account.qbraid.com?gitHubUrl=https://github.com/qosf/monthly-challenges.git)


Quantum computing is used extensively for modelling and solving combinatorial optimisation issues. The purpose of this is to find a problem with binary clauses where the amount of states is immense and, in order to uncover the correct answers, quantum computing produces algorithms of NP-complexity. On the other hand, in quantum computing, we are interested in representing such a model in a quantum circuit and being able to find the optimal states that satisfy the cost function using a classical optimizer.

Multiple companies work around computers and generate an SDK that can generate quantum circuits, in this challenge, we focus on a fundamental step of the Quantum Approximate Optimization Algorithm (QAOA) algorithm, before starting the quantum part one must model a problem in terms of 0 and 1 and convert it into a Quadratic unconstrained binary optimization (QUBO) and this convert it to an Ising model to find the optimal states. To validate the model one makes use of OpenQAOA, an SDK focused on circuitry of the QAOA algorithm. If you want to know more about this SDK you can check the following link https://openqaoa.entropicalabs.com/ 

### Note 

To run on real QPU and simulators use  [qbraid](https://account.qbraid.com/) and send them a dm on their [discord server](https://discord.gg/S99GJBfr) to get credits!

## The Knapsack problem


This challenge is based on the hybrid algorithm QAOA which is based on a variational circuit which will be modified to find the minimum energy by means of a classical optimizer and from the states with the highest probability we will have at least one state that satisfies the conditions of the cost function we use, being based on the knapsack problem.

![QAOA](images/QAOA.png)
<center>Figure 1. General quantum circuit of QAOA </center>



The knapsack problem asks us to find a combination of items such that the total weight is within the capacity of the knapsack and maximize the total value of the items. The Knapsack problem is the simplest nontrivial integer programming model with binary variables, only one constraint and only positive coefficients. 

It is formally defined as follows: We are given an instance of the knapsack problem with the set of items  $N^{\prime}=\{1, \ldots, N\}$, consisting of $N$ items, the $j$-th with profit $p_{j}$ and weight $w_{j}$, and the capacity value $W$. Then, the objective is to select a subset of $N^{\prime}$ such that the total profit of the selected items is maximized and the total weight does not exceed $W$.
$$
\begin{aligned}
\operatorname{maximizar} & \sum_{j=1}^{N} p_{j} x_{j} \\
\text { sujeto a } & \sum_{j=1}^{N} w_{j} x_{j} \leq W \\
& x_{j} \in\{0,1\}, \quad j=1, \ldots, N
\end{aligned}
$$
Let us denote the optimal solution vector by $x^{*}=\left(x_{i}^{*}, \ldots, x_{N}^{*}\right)$ y el valor de solución óptima por $z^{*}$ and the optimal solution value by $z^{*}$. The set $X^{*}$ denotes the optimal solution set, i.e., the set of elements corresponding to the optimal solution vector.

![knapsack problem](images/knapsack_model.png)
<center>Figure 2. Representation of the knapsack model </center>


### Resources

For some guidelines on the objective of this challenge and its passage to a quantum circuit you can consult the following sources, including the original paper explaining this algorithm, as well as some tutorials from different frameworks.

[1] Edward Farhi and Jeffrey Goldstone. (2014). A Quantum Approximate Optimization Algorithm [[4028]](https://arxiv.org/pdf/1411.4028.pdf).

[2] [Qiskit Code](https://learn.qiskit.org/course/ch-applications/solving-combinatorial-optimization-problems-using-qaoa).

[3] [Qiskit Application](https://github.com/qiskit-community/ibm-quantum-challenge-fall-2021/blob/main/solutions-by-authors/challenge-4/challenge-4.ipynb)

[4] [Pennylane Tutorial](https://pennylane.ai/qml/demos/tutorial_qaoa_intro.html)

[5] [Pyquil Tutorial](https://grove-docs.readthedocs.io/en/latest/qaoa.html)

[6] [QUBO's tutorial paper](https://arxiv.org/pdf/1811.11538.pdf)


## Using OpenQAOA

Considering the examples based on OpenQAOA, we already have different classes and methods that facilitate the construction of quantum circuits, but how to generate a QUBO we will rely on docplex. For more information on QAOA examples and how to generate QUBOs, please click [here](https://github.com/entropicalabs/openqaoa/tree/main/examples).

In [1]:
%matplotlib notebook

# Import external libraries to present an manipulate the data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import docplex model to generate the problem to optimize
from docplex.mp.model import Model

# Import the libraries needed to employ the QAOA quantum algorithm using OpenQAOA
from openqaoa import QAOA

# method to covnert a docplex model to a qubo problem
from openqaoa.problems.converters import FromDocplex2IsingModel #check this method and properties
from openqaoa.backends import create_device

# method to find the corrects states for the QAOA boject 
from openqaoa.utilities import ground_state_hamiltonian

In [2]:
values =  [3,6,3,4,5]
weights = [3,5,6,2,4]
max_weight = 13

In [3]:
def Knapsack(values, weights, max_weight):
    
    #init a model
    mdl = Model("Basic Problem using maximize")
    
    # indicate the binary variables 
    n_vars = len(values) # Number of variables
    x = mdl.binary_var_list(n_vars, name="x")  # Using binary variables for this model
    
    # define the objective function
    obj_func = 0
    for i in range(len(weights)):
        obj_func = weights[i]*x[i]
        # obj_func = weights[0]*x[0] + weights[1]*x[1] + weights[2]*x[2] + weights[3]*x[3] + weights[4]*x[4]
    mdl.maximize(obj_func)

    # add  the constraints
    mdl.add_constraint(obj_func <= max_weight)
    
    return  mdl #return model, check FromDocplex2IsingModel

In [4]:
# Converting the Docplex model into its qubo representation
mdl = Knapsack(values, weights, max_weight)
qubo = FromDocplex2IsingModel(mdl)

# Ising encoding of the QUBO problem
ising_encoding = qubo.ising_model  

# Print in a df the ising encoding (we need to remove keys: 'problem_instance' and 'metadata')) 
ising_encoding_dict = ising_encoding.asdict(exclude_keys=['problem_instance', 'metadata'])
pd.DataFrame(ising_encoding_dict)

,terms,weights,constant,n
0,"[4, 5]",10.0,190.5,9
1,"[4, 6]",20.0,190.5,9
2,"[4, 7]",40.0,190.5,9
3,"[8, 4]",60.0,190.5,9
4,"[5, 6]",5.0,190.5,9
5,"[5, 7]",10.0,190.5,9
6,"[8, 5]",15.0,190.5,9
7,"[6, 7]",20.0,190.5,9
8,"[8, 6]",30.0,190.5,9
9,"[8, 7]",60.0,190.5,9


In [5]:
knapsack =  Knapsack(values, weights, max_weight)

# Ising encoding of the QUBO problem for binpacking problem
# Converting the Docplex model of binpacking into its qubo representation
qubo_bin = FromDocplex2IsingModel(knapsack) 

# Ising encoding of the QUBO problem for binpacking problem
ising_encoding_bin = qubo_bin.ising_model

# Docplex encoding of the QUBO problem for binpacking problem
mdl_qubo_docplex = qubo_bin.qubo_docplex

mdl_qubo_docplex.prettyprint()

// This file has been generated by DOcplex
// model name is: Copy of Copy of Basic Problem using maximize
// var contrainer section
dvar bool x[5];
dvar bool slack_C0[4];

// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool x_3;
dvar bool x_4;
dvar bool slack_C0_0;
dvar bool slack_C0_1;
dvar bool slack_C0_2;
dvar bool slack_C0_3;

minimize
 - 524 x_4 - 130 slack_C0_0 - 260 slack_C0_1 - 520 slack_C0_2 - 780 slack_C0_3
 [ 80 x_4^2 + 40 x_4*slack_C0_0 + 80 x_4*slack_C0_1 + 160 x_4*slack_C0_2
 + 240 x_4*slack_C0_3 + 5 slack_C0_0^2 + 20 slack_C0_0*slack_C0_1
 + 40 slack_C0_0*slack_C0_2 + 60 slack_C0_0*slack_C0_3 + 20 slack_C0_1^2
 + 80 slack_C0_1*slack_C0_2 + 120 slack_C0_1*slack_C0_3 + 80 slack_C0_2^2
 + 240 slack_C0_2*slack_C0_3 + 180 slack_C0_3^2 ] + 845;
 
subject to {

}


In [6]:
# Initilize the QAOA object
qaoa = QAOA()

# Set the parameters to work the QAOA algorithm
# where n_shots =1024 and  seed_simulator=1
qaoa.set_backend_properties(n_shots=1024, seed_simulator=1)

#p = 1, a custom type and range from 0 to pi
rep = 1 
qaoa.set_circuit_properties(p=rep, init_type="custom", variational_params_dict={"betas":rep*[0.01*np.pi],"gammas":rep*[0.01*np.pi]})
qaoa.compile(ising_encoding_bin)

# Run the QAOA algorithm
qaoa.optimize()

pd.DataFrame(qaoa.result.lowest_cost_bitstrings(5))

,solutions_bitstrings,bitstrings_energies,probabilities
0,100011101,-4.0,0.001661
1,010011101,-4.0,0.001661
2,000011101,-4.0,0.001661
3,111111101,-4.0,0.001661
4,011111101,-4.0,0.001661


In [7]:
# To find the correct answer using ground_state_hamiltonian
# and  the paremeter is a cost_hamiltonian
correct_solution = ground_state_hamiltonian(qaoa.cost_hamil)
correct_solution

(-4.0,
 ['000011101',
  '100011101',
  '010011101',
  '110011101',
  '001011101',
  '101011101',
  '011011101',
  '111011101',
  '000111101',
  '100111101',
  '010111101',
  '110111101',
  '001111101',
  '101111101',
  '011111101',
  '111111101'])

Valid your answer using docplex, you can see how to check the classical solution using the following tutorial [here](https://github.com/entropicalabs/openqaoa/blob/main/examples/community_tutorials/02_docplex_example.ipynb) 

In [8]:
## docplex solution
sol = mdl_qubo_docplex.solve()
mdl_qubo_docplex.print_solution(print_zeros=True)

objective: -4.000
status: OPTIMAL_SOLUTION(2)
  x_0=0
  x_1=0
  x_2=0
  x_3=0
  x_4=1
  slack_C0_0=1
  slack_C0_1=1
  slack_C0_2=0
  slack_C0_3=1


## Part 2: Improve the quantum circuit

Perform the same process as above now with the variant of using different backends, p values, and different optimizers until you find the one that can provide the correct answers with the least number of iterations, quantum circuit depth.

In [ ]:
## implementation

# Initilize the QAOA object and use a device
device = create_device("local", 'qiskit.aer_simulator')
qaoa = QAOA(device)

# Set the parameters to work the QAOA algorithm
# play with the parameters values
knapsack =  Knapsack(values, weights, max_weight)

# Ising encoding of the QUBO problem for binpacking problem
# Converting the Docplex model of binpacking into its qubo representation
qubo_bin = FromDocplex2IsingModel(knapsack) 

# Ising encoding of the QUBO problem for binpacking problem
ising_encoding_bin = qubo_bin.ising_model

qaoa.set_backend_properties(n_shots=1024, seed_simulator=1)

rep = 5
qaoa.set_circuit_properties(p=rep, init_type="custom", variational_params_dict={"betas":rep*[0.01*np.pi],"gammas":rep*[0.01*np.pi]})
qaoa.compile(ising_encoding_bin)

# Run the QAOA algorithm
qaoa.optimize()

pd.DataFrame(qaoa.result.lowest_cost_bitstrings(5))

## Part 3: Noise Model

The optimal combination that you found with the best optimizer, the lowest number of p's and the correct answer, can give the same answer with noise, use the circuit with a noise model and identify if it gives the same answer.

In [20]:
## implementation using a noise model using qiskit 

## implementation

# Initilize the QAOA object

## real hardware
from qiskit.providers.fake_provider import FakeVigo
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer import QasmSimulator


device = create_device("local", 'qiskit.qasm_simulator')
qaoa = QAOA(device)

# Set the parameters to work the QAOA algorithm
rep = 1
qaoa.set_circuit_properties(p=rep, init_type="custom", variational_params_dict={"betas":rep*[0.01*np.pi],"gammas":rep*[0.01*np.pi]})


qaoa.compile(ising_encoding)

# Run the QAOA algorithm
qaoa.optimize()

pd.DataFrame(qaoa.result.lowest_cost_bitstrings(5))

,solutions_bitstrings,bitstrings_energies,probabilities
0,101111101,-4.0,0.02
1,100111101,-4.0,0.01
2,000111101,-4.0,0.01
3,001011101,-4.0,0.01
4,010111101,-4.0,0.01


## Part 4: New approach

There is a heuristic given by the following [paper](https://arxiv.org/pdf/2211.13914.pdf), try to implement it and identify if it can give the same result

In [ ]:

def KnapsackNewApproach(values,weights, max_weight):


 # indicate the binary variables 

 # define the objective function

 # add  the constraints
    
    # new changes
    return  #return model and check the method FromDocplex2IsingModel and their parameters

In [ ]:
knapsack_new =  KnapsackNewApproach(values,weights, max_weight)

# Ising encoding of the QUBO problem for binpacking problem
ising_encoding_new = knapsack_new.ising_model 

# Docplex encoding of the QUBO problem for binpacking problem
mdl_qubo_docplex_new = knapsack_new.qubo_docplex
mdl_qubo_docplex_new.prettyprint()

In [ ]:
## implementation
## implementation using a noise model using qiskit 

## implementation

# Initilize the QAOA object


device = create_device("local", 'qiskit.qasm_simulator')
qaoa_new = QAOA(device)

# Set the parameters to work the QAOA algorithm

qaoa_new.compile(ising_encoding_new)

# Run the QAOA algorithm
qaoa_new.optimize()

pd.DataFrame(qaoa_new.result.lowest_cost_bitstrings(5))

In [ ]:
correct_solution = 
correct_solution

In [ ]:
## docplex solution
sol = mdl_qubo_docplex_new.solve()
mdl_qubo_docplex_new.print_solution(print_zeros=True)

## Bonus

If you completed the notebook and to arrived at this point, congratulations! You did a great job on learning how to design a QUBO into quantum circuit. Remember that you need an account on qBraid and send us a DM to share the credits and to use a real QPU for this challenge and see the limitation of the NISQ era. You can consider and try to contribute to an important part of the quantum community working in the field of quantum optimization😊

# Acknowledgments

🎉🎉🎉 Special thanks to entropica Labs  for helping us create this challenge and being able to exploit their SDK, OpenQAOA. If you want to know more about OpenQAOA you will see their [discord channel](discord.gg/ana76wkKBd) 🎉🎉🎉 